In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#表示の幅を広げる
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

uketsuke = pd.read_csv('大黒屋様_買取受付一覧 - 受付一覧.csv', low_memory=False)
kaitori = pd.read_csv('出張買取実績_20200819以前.csv', low_memory=False)

In [2]:
#Gitから日本語変換できるライブラリをインストール
!python3.8 -m pip install --upgrade pip
!pip install jaconv
import jaconv

Requirement already up-to-date: pip in /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages (20.2.4)


In [5]:
#結合するために買取情報を編集
#一旦、dictにいれる
L = []
for i in kaitori['姓名（カナ）']:
    a = jaconv.h2z(i)
    L.append(a) 
#'カナ'というカラムを一旦作る
kaitori['カナ'] = '名前'

for i in range(len(L)):
    kaitori['カナ'][i] = L[i]
    

#受付データと買取データ同士で重複している列を買取データ上から削除する
on = ['性別', '生年月日','郵便番号', "姓名（カナ）"]
kaitori.drop(on, axis=1, inplace=True)

#不必要な列を受付データ上から削除する
on = ['Unnamed: 46','Unnamed: 47', 'Unnamed: 48']
uketsuke.drop(on, axis=1, inplace=True)

#"カナ"があるデータで結合する
df0 = pd.merge(uketsuke, kaitori, on='カナ', how='inner')

#必要なカラムだけ書く
on = ['処理状況', '確定日時', '担当', '買取番号', '入電日', '姓（カナ）', '名（カナ）', '姓名', 'カナ', '性別', '年齢', '住所・都道府県', '住所・市町村','建物名/部屋番号','キャンペーンコード', '住所1','住所2', '住所3','出張買取希望日時\n希望日','仕入番号','買取日付','点数','金額','会員番号']
df0 = df0[on]

#データの情報量を確認してみる
def f():
    a=len(kaitori) #出張買取の件数
    b=len(uketsuke) #買取受付の件数
    c=len(df0) #出張買取と買取受付を結合したもの。同一の人が複数あるせいで "a"より行数が多い
    return (a, b, c)
f()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


(112, 733, 134)

## キャンペーンコードの英数字を全て半角に変換

In [6]:
!pip install mojimoji
import mojimoji

In [7]:
#キャンペーンコードの欠損値（聞き取れなかったもの）を０に置き換える
df0['キャンペーンコード'].fillna(value='0', inplace=True)

#キャンペーンコードの数値を半角にする
for num,i in enumerate(df0['キャンペーンコード']):
    b = mojimoji.zen_to_han(i)
    b=b.strip()
    df0["キャンペーンコード"][num]=b

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
print(df0['キャンペーンコード'].isnull().any())

False


In [9]:
df0["キャンペーンコード"][50]

'5ES-AG'

In [10]:
#処理後
df0['キャンペーンコード'].value_counts()

0         61
7EO-CG    22
7EO-CT    19
5ES-AG    10
6EP-AT     4
WEB        4
6EP-AG     2
5ES-AT     2
7EO-CD     2
6EP-BG     2
6FP-BT     1
5ES-BG     1
5EP-BT     1
6EO-BT     1
5EP-AG     1
6EO-AT     1
Name: キャンペーンコード, dtype: int64

In [17]:
#キャンペーンコードが記入されているもののみ抽出
df0['キャンペーンコード'].head()
df0[df0['キャンペーンコード']!= '0']

,処理状況,確定日時,担当,買取番号,入電日,姓（カナ）,名（カナ）,姓名,カナ,性別,年齢,住所・都道府県,住所・市町村,建物名/部屋番号,キャンペーンコード,住所1,住所2,住所3,出張買取希望日時\n希望日,仕入番号,買取日付,点数,金額,会員番号
26,買取成立終了,6/4/2020,佐藤/山野,198,6/3/2020,カネミツ,ミワ,金光 美和,カネミツ ミワ,女性,39,東京都,世田谷区中町3丁目16-13,エミネンス中町206,6EP-AT,世田谷区中町3-16-13エミネンス中町206,NaN,NaN,2020/6/4 12時前後\n平日のみOK\n時間はある程度合わせられるそうです。,4300016572,2020/06/04,10,148000,'5000000000187588
33,買取成立終了,6/5/2020,佐藤/山野,218,6/5/2020,タンジ,リョウ,丹治 竜,タンジ リョウ,男性,40,東京都,目黒区八雲3-11-5,パークアクシス自由が丘テラス401号室,6EP-BG,目黒区八雲3-11-5-401,NaN,NaN,6/5 何時でも\n6/6 何時でも\n6/7 何時でも,4300016568,2020/06/05,1,3300,'5000000000187585
35,買取成立終了,6/8/2020,佐藤/山野,223,6/6/2020,イイノ,カズコ,飯野 和子,イイノ カズコ,女性,74,東京都,目黒区大岡山1-31-17,大岡山シティハウス403号室,6EP-AT,目黒区大岡山1-31-17,NaN,NaN,6/8 午後から\n6/9 夕方から,4300016576,2020/06/08,1,1100,'5000000000187579
36,買取成立終了,6/8/2020,佐藤/山野,235,6/8/2020,ミウラ,アキオ,三浦 昭雄,ミウラ アキオ,男性,85,東京都,世田谷区等々力2-9-11,金山コーポ303,6EP-AT,世田谷区等々力2-9-11-303,NaN,NaN,いつでも,4300016577,2020/06/08,1,1100,'5000000000187578
37,買取成立終了,6/9/2020,佐藤/山野,210,6/4/2020,サワダ,カナ,澤田 香菜,サワダ カナ,女性,37,東京都,NaN,港南第三アパート825号,5ES-BG,港区港南4丁目2-4-825,NaN,NaN,6/9もしくは6/11\n6/17もしくは6/19 17時まで,4300016587,2020/06/09,6,185000,'5000000000187577
45,でない,6/16/2020,佐藤,290,6/16/2020,オオタケ,ノゾミ,大竹 望,オオタケ ノゾミ,女性,40,東京都,葛飾区奥戸3-9-9,村越コーポ201,5ES-AG,葛飾区奥戸3-9-9,NaN,NaN,6/16 16:30まで,6600000030,2020/07/04,4,550,'5000000000187574
46,でない,6/16/2020,佐藤,290,6/16/2020,オオタケ,ノゾミ,大竹 望,オオタケ ノゾミ,女性,40,東京都,葛飾区奥戸3-9-9,村越コーポ201,5ES-AG,葛飾区奥戸3-9-9,NaN,NaN,6/16 16:30まで,6600000010,2020/06/21,8,33000,'5000000000187574
47,でない,6/16/2020,佐藤,290,6/16/2020,オオタケ,ノゾミ,大竹 望,オオタケ ノゾミ,女性,40,東京都,葛飾区奥戸3-9-9,村越コーポ201,5ES-AG,葛飾区奥戸3-9-9,NaN,NaN,6/16 16:30まで,6600000002,2020/06/12,7,3300,'5000000000187574
48,キャンセル,6/16/2020,コールセンター,269,6/13/2020,オオタケ,ノゾミ,大竹 望,オオタケ ノゾミ,女性,40,東京都,葛飾区奥戸3-9-9,村越コーポ201,5ES-AG,葛飾区奥戸3-9-9,NaN,NaN,6/14 午前10:30～11時頃,6600000030,2020/07/04,4,550,'5000000000187574
49,キャンセル,6/16/2020,コールセンター,269,6/13/2020,オオタケ,ノゾミ,大竹 望,オオタケ ノゾミ,女性,40,東京都,葛飾区奥戸3-9-9,村越コーポ201,5ES-AG,葛飾区奥戸3-9-9,NaN,NaN,6/14 午前10:30～11時頃,6600000010,2020/06/21,8,33000,'5000000000187574


## チラシの設定されているコードを読み取る

In [24]:
#配布時に設定したチラシの設定コード
code0 = pd.read_excel('/Users/satoshiido/Downloads/大黒屋/オフライン - チラシ/チラシコード一覧_20201028.xlsx')
code0.head()

,実施月,エリア,メディア,CR,訴求軸,キャンペーンコード
0,折込5月22日（金）ポスティング5月23日（土）～29日（金）,東日本（EAST),折込（Orikomi),A,現金お届け（Genkin）,5EO-AG
1,折込5月22日（金） ポスティング5月23日（土）～29日（金）,東日本（EAST),折込（Orikomi),A,宅配買取（Takuhai）,5EO-AT
2,折込5月22日（金） ポスティング5月23日（土）～29日（金）,東日本（EAST),折込（Orikomi),B,現金お届け（Genkin）,5EO-BG
3,折込5月22日（金） ポスティング5月23日（土）～29日（金）,東日本（EAST),折込（Orikomi),B,宅配買取（Takuhai）,5EO-BT
4,折込5月22日（金） ポスティング5月23日（土）～29日（金）,東日本（EAST),ポスティング（Posting),A,現金お届け（Genkin）,5EP-AG


In [26]:
#チラシの設定コードの数値を半角にする
for num,i in enumerate(code0['キャンペーンコード']):
    b = mojimoji.zen_to_han(i)
    b=b.strip()
    code0["キャンペーンコード"][num]=b

,実施月,エリア,メディア,CR,訴求軸,キャンペーンコード
0,折込5月22日（金）ポスティング5月23日（土）～29日（金）,東日本（EAST),折込（Orikomi),A,現金お届け（Genkin）,5EO-AG
1,折込5月22日（金） ポスティング5月23日（土）～29日（金）,東日本（EAST),折込（Orikomi),A,宅配買取（Takuhai）,5EO-AT
2,折込5月22日（金） ポスティング5月23日（土）～29日（金）,東日本（EAST),折込（Orikomi),B,現金お届け（Genkin）,5EO-BG
3,折込5月22日（金） ポスティング5月23日（土）～29日（金）,東日本（EAST),折込（Orikomi),B,宅配買取（Takuhai）,5EO-BT
4,折込5月22日（金） ポスティング5月23日（土）～29日（金）,東日本（EAST),ポスティング（Posting),A,現金お届け（Genkin）,5EP-AG
5,折込5月22日（金） ポスティング5月23日（土）～29日（金）,東日本（EAST),ポスティング（Posting),A,宅配買取（Takuhai）,5EP-AT
6,折込5月22日（金） ポスティング5月23日（土）～29日（金）,東日本（EAST),ポスティング（Posting),B,現金お届け（Genkin）,5EP-BG
7,折込5月22日（金） ポスティング5月23日（土）～29日（金）,東日本（EAST),ポスティング（Posting),B,宅配買取（Takuhai）,5EP-BT
8,折込5月22日（金） ポスティング5月23日（土）～29日（金）,東日本（EAST),店舗配布用（Shop),A,現金お届け（Genkin）,5ES-AG
9,折込5月22日（金） ポスティング5月23日（土）～29日（金）,東日本（EAST),店舗配布用（Shop),A,宅配買取（Takuhai）,5ES-AT
